In [31]:
from collections import Counter

def get_suffix_array(string):
    '''
    generates suffix array from string
    '''
    return(list(sorted(range(len(string)), key=lambda i:string[i:])))


def get_bwt(string, s_array=None):
    '''
    returns burrows-wheeler transform of given string
    '''
    if s_array is None:
        s_array = get_suffix_array(string)
    return("".join(string[idx - 1] for idx in s_array))


def get_occurences(bwt, letters=None):
    '''
    generates occurences array from bwt
    '''
    if letters is None:
        letters = set(bwt)

    result = {letter:[0] for letter in letters}
    result[bwt[0]] = [1]
    for letter in bwt[1:]:
        for i, j in result.items():
            j.append(j[-1] + (i == letter))
    return(result)


def get_count(string, letters=None):
    '''
    generate count array
    '''
    count = 0
    result = {}

    if letters is None:
        letters = set(string)

    c = Counter(string)

    for letter in sorted(letters):
        result[letter] = count
        count += c[letter]
    return result


def find_word(word, text, alphabet=None):
    '''
    find word in text using Burrows-Wheeler transform, Suffix array and
    count and occurences arrays
    '''
    if not set(word) <= set(text):
        return []

    sa = get_suffix_array(text)
    bwt = get_bwt(text, sa)
    cnt = get_count(text)
    occ = get_occurences(bwt)
    if alphabet is None:
        alphabet = sorted(set(text))

    # initialization
    a = word[-1]
    F = cnt[a]
    print(alphabet, cnt)
    L = cnt[alphabet[alphabet.index(a) + 1]] - 1

    # finding first and last suffixes starting with word
    for i in range(len(word)-2, -1, -1):
        a = word[i]
        F = cnt[a] + occ[a][F-1]
        L = cnt[a] + occ[a][L] - 1

    # using suffix array find indexes of word occurrences in the text
    matches = []
    for i in range(F, L+1):
        matches.append(sa[i])
    matches.sort()

    return matches

In [25]:
word = 'ANA'
text = 'BANANA$'

sa = get_suffix_array(text)
bwt = get_bwt(text)
occ = get_occurences(bwt)
cnt = get_count(text)
matches = find_word(word, text)

print('Suffix array: ', sa)
print('burrows-wheeler transform of text: ', bwt)
print('occurences array: ', occ)
print('counts array: ', cnt)
print('matches of ANA in BANANA: ', matches)

Suffix array:  [6, 5, 3, 1, 0, 4, 2]
burrows-wheeler transform of text:  ANNB$AA
occurences array:  {'B': [0, 0, 0, 1, 1, 1, 1], '$': [0, 0, 0, 0, 1, 1, 1], 'A': [1, 1, 1, 1, 1, 2, 3], 'N': [0, 1, 2, 2, 2, 2, 2]}
counts array:  {'$': 0, 'A': 1, 'B': 4, 'N': 5}
matches of ANA in BANANA:  [1, 3]


In [37]:
matches = find_word(word='ISSI', text='MISSISSIPI$')
print('matches of ISSI in MISSISSIPI: ', matches)

['$', 'I', 'M', 'P', 'S'] {'$': 0, 'I': 1, 'M': 5, 'P': 6, 'S': 7}
matches of ISSI in MISSISSIPI:  [1, 4]


In [29]:
matches = find_word(word='A', text='B$')
print('matches of A in BB: ', matches)

matches of A in BB:  []
